In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from math import nan
from sqlalchemy import create_engine
import pyodbc
import os
import re
import mysql.connector

In [5]:
os.getcwd()

'D:\\dataFCNOV'

In [4]:
os.chdir('D:\dataFCNOV')

In [7]:
mydb = mysql.connector.connect(
  host="103.27.62.149",
  user="y4a_sc_intern",
  password="68546771",
  database="sc_main"
)



In [11]:
mycursor = mydb.cursor()

mycursor.execute("""
SELECT dim.*, rs.sale, ds.daily_sale, amz.inv_amz, yes.inv_y4a,
CASE WHEN ((IFNULL(amz.inv_amz,0) +  IFNULL(yes.inv_y4a,0)) <= 3*ds.daily_sale and rs.sale is null)  THEN (ds.daily_sale*7)
ELSE IFNULL(rs.sale,0) END QNT_REMOVE_OOS

FROM (
# BASE DATA
SELECT DISTINCT d.`week`, A.sku
FROM dim_date d
CROSS JOIN (SELECT DISTINCT  sku
FROM customer_order_by_ship_date
WHERE sku IS NOT NULL and shipped_date > DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 26 WEEK)) A
WHERE d.date between DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 26 WEEK) AND DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 1 WEEK)
) dim

# REALSALES
LEFT JOIN
(
SELECT d.`week`, s.sku, SUM(s.qty) as sale
FROM customer_order_by_ship_date s
LEFT JOIN dim_date d on d.date = s.shipped_date
WHERE s.sku IS NOT NULL and s.shipped_date > DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 26 WEEK)
GROUP BY d.`week`, s.sku
) rs ON dim.`week` = rs.`week` and dim.sku = rs.sku

#daily sale
LEFT JOIN 
(
SELECT sku, avg(qty) as daily_sale
FROM customer_order_by_ship_date 
WHERE SKU IS NOT NULL AND shipped_date> DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 26 WEEK)
GROUP BY 1
) ds ON dim.sku = ds.sku


# AMZ INVENTORY
LEFT JOIN
(
SELECT d1.`week` as log_week, ia.asin, p.sku, AVG(ia.inventory_manufacturing) as inv_amz
FROM inventory_amazon ia
LEFT JOIN dim_date d1 on d1.date = ia.log_date
LEFT JOIN dim_product p on p.asin = ia.asin
WHERE ia.log_date > DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 26 WEEK)
AND week(ia.log_date) < week(CURDATE())
GROUP BY d1.`week`, ia.asin, p.sku
) amz ON amz.log_week = dim.`week` and amz.sku = dim.sku

# YES INVENTORY
LEFT JOIN
(
SELECT d2.`week`, iy.sku, avg(iy.total) as inv_y4a
FROM inventory_yes4all iy
LEFT JOIN dim_date d2 on d2.date = iy.log_date
WHERE iy.log_date > DATE_SUB(DATE_SUB(CURDATE(), INTERVAL DAYOFWEEK(CURDATE())-1 DAY),INTERVAL 26 WEEK)
AND week(iy.log_date) < week(CURDATE())
GROUP BY d2.`week`, iy.sku
) yes ON dim.sku = yes.sku and dim.`week` = yes.`week`



""")

myresult = mycursor.fetchall()

In [12]:
column_name = [i[0] for i in mycursor.description]
column_name

['week', 'sku', 'sale', 'daily_sale', 'inv_amz', 'inv_y4a', 'QNT_REMOVE_OOS']

In [15]:
raw = pd.DataFrame(myresult, columns = column_name)
raw

,week,sku,sale,daily_sale,inv_amz,inv_y4a,QNT_REMOVE_OOS
0,19,03NF,47,10.3553,95.5000,1867.1667,47.0000
1,20,03NF,62,10.3553,47.1429,1754.2857,62.0000
2,21,03NF,46,10.3553,38.0000,1982.0000,46.0000
3,22,03NF,59,10.3553,37.8571,2082.0000,59.0000
4,23,03NF,84,10.3553,54.1429,1860.7143,84.0000
...,...,...,...,...,...,...,...
26619,40,HUJZ,None,2.0000,None,80.0000,0.0000
26620,41,HUJZ,None,2.0000,None,80.0000,0.0000
26621,42,HUJZ,None,2.0000,None,80.0000,0.0000
26622,43,HUJZ,None,2.0000,None,80.0000,0.0000


In [20]:
table2 = mydb.cursor()

table2.execute("""
SELECT sku, product_name , company, life_cycle, vendor_actual FROM `dim_product`
""")

info = table2.fetchall()

In [21]:
column_info = [i[0] for i in table2.description]
column_info

['sku', 'product_name', 'company', 'life_cycle', 'vendor_actual']

In [22]:
info = pd.DataFrame(info, columns = column_info)
info

,sku,product_name,company,life_cycle,vendor_actual
0,01JV,Supply WH - Brown Box - Dumbbell Neoprene - 10...,Y4A,WH Supply - Temp,Jason
1,02MN,Weight Plate 1in - 10 lbs - 4 pcs,Y4A,Normal,"GY,Jason"
2,03G0,Stability Wooden Rocker Board - Red,Y4A,Unknown,None
3,03NF,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal,"GY,Roger"
4,07AG,Agility Hex Ring - 12 Rings,Y4A,Normal,Jason
...,...,...,...,...,...
3708,ZZE3,Dumbbell Adjustable - 200 lbs - Package 3,Y4A,Normal,"Jason,Leeton"
3709,ZZU4,Bow Resistance Band,Y4A,Withdrawal,Roger
3710,ZZXT,Dumbbell PVC - 12 lbs - Emeral Green - Single,Y4A,Inactived,Jason
3711,ZZZH,Axe H307 Black + Knife H153B,Y4A,Withdrawal,None


In [23]:
full_data = pd.merge(raw, info[['sku','product_name','company','life_cycle']], on = 'sku', how = 'left')
full_data.head()

,week,sku,sale,daily_sale,inv_amz,inv_y4a,QNT_REMOVE_OOS,product_name,company,life_cycle
0,19,03NF,47,10.3553,95.5000,1867.1667,47.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
1,20,03NF,62,10.3553,47.1429,1754.2857,62.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
2,21,03NF,46,10.3553,38.0000,1982.0000,46.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
3,22,03NF,59,10.3553,37.8571,2082.0000,59.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
4,23,03NF,84,10.3553,54.1429,1860.7143,84.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal


In [26]:
full_data = full_data[~full_data['life_cycle'].isnull()]

In [28]:
full_data[full_data['life_cycle'].isnull()]

,week,sku,sale,daily_sale,inv_amz,inv_y4a,QNT_REMOVE_OOS,product_name,company,life_cycle


In [30]:
full_data

,week,sku,sale,daily_sale,inv_amz,inv_y4a,QNT_REMOVE_OOS,product_name,company,life_cycle
0,19,03NF,47,10.3553,95.5000,1867.1667,47.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
1,20,03NF,62,10.3553,47.1429,1754.2857,62.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
2,21,03NF,46,10.3553,38.0000,1982.0000,46.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
3,22,03NF,59,10.3553,37.8571,2082.0000,59.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
4,23,03NF,84,10.3553,54.1429,1860.7143,84.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
...,...,...,...,...,...,...,...,...,...,...
26619,40,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26620,41,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26621,42,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26622,43,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction


In [54]:
#filter out mask and HMD
df = full_data[(~full_data['product_name'].str.contains('Mask')) & (~full_data['company'].str.contains('HMD'))]
df

,week,sku,sale,daily_sale,inv_amz,inv_y4a,QNT_REMOVE_OOS,product_name,company,life_cycle
0,19,03NF,47,10.3553,95.5000,1867.1667,47.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
1,20,03NF,62,10.3553,47.1429,1754.2857,62.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
2,21,03NF,46,10.3553,38.0000,1982.0000,46.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
3,22,03NF,59,10.3553,37.8571,2082.0000,59.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
4,23,03NF,84,10.3553,54.1429,1860.7143,84.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
...,...,...,...,...,...,...,...,...,...,...
26619,40,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26620,41,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26621,42,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26622,43,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction


In [55]:
#filter life cycle
lc = ['Turnkey','Normal','1st year Growth', 'Introduction','Withdrawal','Local Only - Temp']
df = df[df['life_cycle'].isin(lc)]
df

,week,sku,sale,daily_sale,inv_amz,inv_y4a,QNT_REMOVE_OOS,product_name,company,life_cycle
0,19,03NF,47,10.3553,95.5000,1867.1667,47.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
1,20,03NF,62,10.3553,47.1429,1754.2857,62.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
2,21,03NF,46,10.3553,38.0000,1982.0000,46.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
3,22,03NF,59,10.3553,37.8571,2082.0000,59.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
4,23,03NF,84,10.3553,54.1429,1860.7143,84.0000,Combo Agility Ladder Yellow 12 Rung + Agility ...,Y4A,Normal
...,...,...,...,...,...,...,...,...,...,...
26619,40,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26620,41,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26621,42,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction
26622,43,HUJZ,None,2.0000,None,80.0000,0.0000,Dumbbell - Urethane - 40lbs,Y4A,Introduction


In [56]:
df['QNT_REMOVE_OOS'] = df['QNT_REMOVE_OOS'].astype(int)

<ipython-input-56-25b628418849>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QNT_REMOVE_OOS'] = df['QNT_REMOVE_OOS'].astype(int)


In [57]:
df['QNT_REMOVE_OOS']

0        47
1        62
2        46
3        59
4        84
         ..
26619     0
26620     0
26621     0
26622     0
26623     0
Name: QNT_REMOVE_OOS, Length: 22334, dtype: int32

In [58]:
df = pd.pivot_table(df,values='QNT_REMOVE_OOS', index = ['week'], columns = ['sku'])
df

sku,02MN,03NF,07AG,08KY,09SM,0ELF,0JJV,0M61,1ARL,1J6H,...,ZS2H,ZSM8,ZSQM,ZT2X,ZVTJ,ZW2S,ZXSV,ZYLU,ZZCE,ZZU4
week,,,,,,,,,,,,,,,,,,,,,
19,120,47,55,0,35,0,83,0,6,35,...,0,12,17,4,86,0,157,1,85,1
20,120,62,20,0,29,0,239,0,16,26,...,6,2,17,8,78,12,286,19,113,0
21,120,46,12,0,31,2,151,6,16,57,...,48,12,17,14,75,34,275,19,47,1
22,120,59,32,0,33,0,176,6,3,57,...,102,5,17,16,62,0,213,37,1,2
23,120,84,36,1,83,0,109,2,24,70,...,139,12,17,26,51,0,245,14,74,1
24,120,88,36,1,48,0,66,10,13,97,...,143,12,17,32,10,0,598,25,74,2
25,120,97,30,0,82,0,58,10,25,105,...,146,1,17,6,18,70,702,29,43,7
26,120,87,39,0,63,0,46,10,31,37,...,137,12,0,3,20,70,424,23,76,0
27,120,95,47,0,80,0,75,10,23,49,...,122,1,0,4,15,70,174,27,74,2


In [59]:
df = pd.DataFrame(df.to_records())
df

,week,02MN,03NF,07AG,08KY,09SM,0ELF,0JJV,0M61,1ARL,...,ZS2H,ZSM8,ZSQM,ZT2X,ZVTJ,ZW2S,ZXSV,ZYLU,ZZCE,ZZU4
0,19,120,47,55,0,35,0,83,0,6,...,0,12,17,4,86,0,157,1,85,1
1,20,120,62,20,0,29,0,239,0,16,...,6,2,17,8,78,12,286,19,113,0
2,21,120,46,12,0,31,2,151,6,16,...,48,12,17,14,75,34,275,19,47,1
3,22,120,59,32,0,33,0,176,6,3,...,102,5,17,16,62,0,213,37,1,2
4,23,120,84,36,1,83,0,109,2,24,...,139,12,17,26,51,0,245,14,74,1
5,24,120,88,36,1,48,0,66,10,13,...,143,12,17,32,10,0,598,25,74,2
6,25,120,97,30,0,82,0,58,10,25,...,146,1,17,6,18,70,702,29,43,7
7,26,120,87,39,0,63,0,46,10,31,...,137,12,0,3,20,70,424,23,76,0
8,27,120,95,47,0,80,0,75,10,23,...,122,1,0,4,15,70,174,27,74,2
9,28,120,47,69,0,123,0,134,10,6,...,153,8,0,6,8,70,315,49,74,0


In [48]:
def find_new_x (x,j,col):
    z = 3
    done = False
    mean = df[col].mean()
    std = df[col].std()
    while done == False:
        x = mean + z*std
        df[col][j] = x
        if df[col].std() == 0:
            z = z - 0.25
        else:
            z_new = (x - df[col].mean())/df[col].std()
            if z_new >= 3:
                z = z - 0.25
            else:
                done = True
    return(x)

In [49]:
def find_new_x_n (x,j,col):
    z = -3
    done = False
    mean = df[col].mean()
    std = df[col].std()
    while done == False:
        x = mean + z*std
        df[col][j] = x
        if df[col].std() == 0:
            z = z + 0.25
        else:
            z_new = (x - df[col].mean())/df[col].std()
            if z_new <= -3:
                z = z + 0.25
            else:
                done = True
    return(x)

In [62]:
def cleancol(col):
    restart = True
    while restart:
        restart = False
        if (df[col].std() == 0) or (df[col].nunique() <= 0.3*len(df)):
            df[col] = df[col]
        else:
            for i in range(0,len(df)):
                if ((df.iloc[i,df.columns.get_loc(col)]- df[col].mean())/df[col].std()) > 3:
                    df.iloc[i,df.columns.get_loc(col)] = find_new_x(df[col][i],i,col)
                    restart = True
                    break
                elif ((df.iloc[i,df.columns.get_loc(col)]- df[col].mean())/df[col].std()) < -3:
                    df.iloc[i,df.columns.get_loc(col)] = find_new_x_n(df[col][i],i,col)
                    restart = True
                    break
                else:
                    df.iloc[i,df.columns.get_loc(col)] = df.iloc[i,df.columns.get_loc(col)]
    return df[col]

In [52]:
df['WNLG'].nunique()

2

In [63]:
for q in df.columns[1:]:
    print(q)
    cleancol(q)

02MN
03NF
07AG
08KY
09SM
0ELF
0JJV
0M61
1ARL
1J6H
1NHS
1RNH
1XTU
1YHQ
2JH2
2KLC
2UBN
36DB
37XZ
399P
3D9D
3KHB


<ipython-input-48-b64ff87a7221>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][j] = x


3MX1
3MXN
3VSD
3W51
3Y12
3YZV
3ZMT
45MH
4M82
4YHG
4Z9X
51F7
5AF4
5CH9
5KF7
5NRP
5SA0
5SU6
60DA
63ML
64TL
673J
6B9Q
6CLE
6KWK
6LWD
6RX7
6V3G
6Y86
7BW4
7CPK
7FVT
7KMQ
7MZ2
7RBK
7VBX
7XU4
7ZKK
8ACX
8BX6
8F0P
8GHV
8JD3
8K3B
8NFL
8XQ1
8YCR
91VY
9E00
9KSP
9PRY
9RNM
9RV3
9SWE
9T3D
9TZN
9UAZ
9VRG
A0QD
A4BN
A8QC
A9CZ
A9Q1
AAAG
AAAH
AAAI
AAAR
AAAS
AAAT
AAAU
AAAV
AC1G
AC42
AEA6
AJJM
AJSG
AKHG
AKNK
AKU5
AMNS
ANCT
ANCU
APMB
AQ16
AS5W
ASCJ
AVMH
AWNY
AXW9
AY14
AYGZ
AYPK
AZ6B
B0AL
B0RJ
B1BQ
B3HD
B42T
B62R
B6D1
BAAA
BAAB
BAAC
BAAD
BAAE
BAAH
BAAI
BAAN
BAAW
BAAX
BACD
BCSS
BDA1
BDM8
BDXR
BK27
BKVQ
BL0J
BLC7
BME4
BNRE
BOE5
BP5K
BQG1
BW0X
BW7V
BWRM
BXGM
BXJ4
BZ4G
C1AW
C1KK
C2K6
C2KR
C444
C5CT
C5TJ
C756
C84X
C954
CAAH
CAAI
CAWR
CAY1
CCA8
CCZN
CFCN
CFE9
CFH8
CFL8
CGAS
CKBG
CKGO
CKT4
CLHZ
CLUK
CM5X
CMJ4
CMX1
CNMF
CREG
CTZ7
CX7L
CXCM
CXKW
CXMR
CZB5
D1IB
D1MY
D25M
D297
D2CL
D377
D37R
D5XI
D68N
D68S
D6RW
D7SL
D8FC
D8U0
D8UJ
D9YB
DA0G
DABM
DAT7
DB6F
DCPL
DFPW
DFTC
DJ3U
DKR2
DL2Z
DMHY
DNC8
DOHL
DS24
DSAX
DU1T
DU7Y


In [64]:
df

,week,02MN,03NF,07AG,08KY,09SM,0ELF,0JJV,0M61,1ARL,...,ZS2H,ZSM8,ZSQM,ZT2X,ZVTJ,ZW2S,ZXSV,ZYLU,ZZCE,ZZU4
0,19,120,47,55,0,35,0,83,0,6,...,0,12,17,4.000000,86,0,157.000000,1,85,1
1,20,120,62,20,0,29,0,239,0,16,...,6,2,17,8.000000,78,12,286.000000,19,113,0
2,21,120,46,12,0,31,2,151,6,16,...,48,12,17,14.000000,75,34,275.000000,19,47,1
3,22,120,59,32,0,33,0,176,6,3,...,102,5,17,16.000000,62,0,213.000000,37,1,2
4,23,120,84,36,1,83,0,109,2,24,...,139,12,17,26.000000,51,0,245.000000,14,74,1
5,24,120,88,36,1,48,0,66,10,13,...,143,12,17,25.698483,10,0,598.000000,25,74,2
6,25,120,97,30,0,82,0,58,10,25,...,146,1,17,6.000000,18,70,646.463359,29,43,7
7,26,120,87,39,0,63,0,46,10,31,...,137,12,0,3.000000,20,70,424.000000,23,76,0
8,27,120,95,47,0,80,0,75,10,23,...,122,1,0,4.000000,15,70,174.000000,27,74,2
9,28,120,47,69,0,123,0,134,10,6,...,153,8,0,6.000000,8,70,315.000000,49,74,0


In [65]:
df.to_csv('cleaned_data_Nov.csv',index = False)